In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer,StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import math

In [2]:
def standard(train,test):
    scaler = StandardScaler().fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train,test

In [9]:
x_train = pd.read_csv('../data/x_train_40.csv')
y_train = pd.read_csv('../data/pre_y_train.csv')
x_test = pd.read_csv('../data/x_test_40.csv')
x_test_id = pd.read_csv('../data/pre_x_test.csv',usecols=['id'])
#x_train,x_test = standard(x_train,x_test)
#print x_train
#for i in x_train.columns:
#    if x_train[i].dtype != np.int64 and x_train[i].dtype != np.float64:
#        print i

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

In [5]:
params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['eta'] = 0.02
params['max_depth'] = 6
d_train = xgb.DMatrix(x_train,y_train)
d_valid = xgb.DMatrix(x_valid,y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 1000, watchlist ,early_stopping_rounds=50, verbose_eval=10)

[0]	train-rmse:14.8197	valid-rmse:14.8184
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[10]	train-rmse:12.113	valid-rmse:12.1124
[20]	train-rmse:9.90217	valid-rmse:9.90198
[30]	train-rmse:8.09661	valid-rmse:8.0969
[40]	train-rmse:6.62237	valid-rmse:6.62287
[50]	train-rmse:5.41906	valid-rmse:5.41975
[60]	train-rmse:4.43738	valid-rmse:4.43822
[70]	train-rmse:3.63704	valid-rmse:3.63847
[80]	train-rmse:2.98514	valid-rmse:2.98701
[90]	train-rmse:2.45488	valid-rmse:2.45762
[100]	train-rmse:2.02447	valid-rmse:2.02828
[110]	train-rmse:1.67618	valid-rmse:1.68121
[120]	train-rmse:1.39558	valid-rmse:1.40187
[130]	train-rmse:1.17099	valid-rmse:1.17844
[140]	train-rmse:0.992494	valid-rmse:1.00138
[150]	train-rmse:0.852192	valid-rmse:0.862973
[160]	train-rmse:0.743445	valid-rmse:0.756268
[170]	train-rmse:0.66027	valid-rmse:0.675332
[180]	train-rmse:0.597878	valid-rmse:0.615311
[190]	train-rmse:0.55167

In [6]:
d_test = xgb.DMatrix(x_test)
y_test = bst.predict(d_test)

In [7]:
sub = pd.DataFrame()
sub['id'] = np.array(x_test_id['id'].values,np.int32)
sub['price_doc'] = np.exp(y_test)-1.0
sub.to_csv('../output/simple10.csv', index=False)